In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [12]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/facc/facc_data/')

In [15]:
import cv2
import numpy as np

def load_images(folder):
  images = []
  for filename in os.listdir(folder):
    img = cv2.imread(os.path.join(folder, filename))
    if img is not None:
      img = cv2.resize(img, (256, 256))
      img = img / 255.
      images.append(img)
  return np.array(images)

In [13]:
!ls

fishandchipimgs  notfishandchipimgs


In [16]:
fac = load_images('fishandchipimgs')
not_fac = load_images('notfishandchipimgs')

In [18]:
X = np.concatenate((fac, not_fac), axis=0)
y = np.zeros((X.shape[0]))
y[:fac.shape[0]] = 1

In [19]:
print(X.shape)
print(y.shape)

(148, 256, 256, 3)
(148,)


In [28]:
X_flattened = X.reshape(X.shape[0], 256 * 256 * 3)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, test_size = 0.2)

In [32]:
from sklearn.svm import LinearSVC
linclf = LinearSVC()
linclf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [33]:
from sklearn.metrics import classification_report
y_pred = linclf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.56      0.42      0.48        12
         1.0       0.67      0.78      0.72        18

    accuracy                           0.63        30
   macro avg       0.61      0.60      0.60        30
weighted avg       0.62      0.63      0.62        30



In [34]:
import pickle
pickle.dump(linclf, open('facc_model.pkl', 'wb'))